# Objectives: 
## - Develop a predictive (regression) model that predicts prices of (future) iPhone releases
## - Train the model with data of iPhone prices from first release (2007) to present day
## - Deploy predictive model into the front-end as a web application using Dash (Python) and/or React (JavaScript) 


In [190]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import os 
import seaborn as sns 
%matplotlib inline 

data_path = "./iphone_releases.csv"

iphones = pd.read_csv(data_path)

def saveOutput(df, filename):
    df.to_csv(filename)

### Stage 1: Pre-Processing
#### - Clean, format, and repurpose iPhone price data for the predictive model

In [191]:
## DATA CLEANING AND REFORMATTING 

# CONVERT ALL TO LOWERCASE 
# turn rows to uppercase 
iphones = iphones.apply(lambda col: col.str.upper() if 'O' == col.dtype  else col) 
# turn columns to uppercase 
iphones.columns = iphones.columns.str.upper() 

# REFORMAT "DATE" COLUMN 
# exclude month and day, only keep year 
iphones["DATE"] = iphones["DATE"].str[:4] 

# RENAME COLUMNS 
# rename "date" column to "year" 
iphones.rename(columns={"DATE": "YEAR"}, inplace=True) 
# rename "memory" column to "GB" 
iphones.rename(columns={"MEMORY": "GB"}, inplace=True) 

# FILL MISSING VALUES 
# set the "version" of the first 2 rows (original iPhone) as "1" 
iphones.loc[2007 == iphones["YEAR"].astype(int) , "VERSION"] = 1 

# set missing "edition" values to 0, for iPhones not of special edition 
edition_regular = "STANDARD" 
iphones.loc[iphones["EDITION"].isnull(), "EDITION"] = edition_regular 

# set missing "scale" values to "STANDARD", for iPhones not of special size 
size_regular = "MEDIUM" 
iphones.loc[iphones["SCALE"].isnull(), "SCALE"] = size_regular 

# RE-FORMAT SIMILAR (REDUNDANT) VALUE TYPES 
# reformat "edition" column, merge "S" and "PRO" values 
edition_special = "SPECIAL" 
iphones.loc[('S' == iphones["EDITION"]) | ("PRO" == iphones["EDITION"]), "EDITION"] = edition_special 

# reformat "scale" column, merge "Plus" and "Max" values 
size_large = "LARGE" 
iphones.loc[("PLUS" == iphones["SCALE"]) | ("MAX" == iphones["SCALE"]), "SCALE"] = size_large 

# REMOVE OUTLIERS / EXCEPTIONAL DATA POINTS 
# drop samples with "edition" values other than "SPECIAL" or "STANDARD" (like "R" or "C") 
iphones = iphones.loc[~((iphones["EDITION"] != edition_regular) & (iphones["EDITION"] != edition_special))] 

# drop samples with "edition" values other than "NORMAL" or "LARGE" (like "MINI") 
iphones = iphones.loc[~((iphones["SCALE"] != size_regular) & (iphones["SCALE"] != size_large))] 

# drop releases from 2007 due to unusual prices 
iphones = iphones.loc[~(2007 == iphones["YEAR"].astype(int))]

In [192]:
## DATA ENGINEERING (REPURPOSING) 

# convert "EDITION" to binary feature (via casting) 
iphones.loc[:, "EDITION"] = (edition_special == iphones["EDITION"]).astype(int) 
# rename "EDITION" column  to "SPECIAL" 
iphones.rename(columns={"EDITION": "SPECIAL"}, inplace=True) 

# convert "SCALE" to binary feature (via casting) 
iphones.loc[:, "SCALE"] = (size_large == iphones["SCALE"]).astype(int) 
# rename "SCALE" column to "LARGE" 
iphones.rename(columns={"SCALE": "LARGE"}, inplace=True) 


## **EXPERIMENTAL** 
# GET DUMMIES FOR DIFFERENT STORAGE CAPACITY VALUES 
# although numerical, gigabyte capacities come in fixed (non-continuous) values (powers of 2) and don't seem to linearly affect the price 
# essentially, they are more categorical in nature, and only represented numerically 
iphones = pd.concat( 
    [
        iphones, 
        pd.get_dummies( 
            iphones["GB"], 
            prefix="GB", 
            drop_first=True 
        ).astype(int) 
    ], 
    axis=1 
) 
# drop "GB" column 
iphones.drop(["GB"], axis=1, inplace=True) 

### Stage 2: EDA (Exploratory Data Analysis) 

In [193]:
## DATA VISUALIZATION 



In [194]:
saveOutput(iphones, "iphones_output.csv")

iphones

YEAR VERSION SPECIAL LARGE   PRICE  GB_8  GB_16  GB_32  GB_64  GB_128  \
0    2007       1       0     0   499.0     0      0      0      0       0   
1    2007       1       0     0   599.0     1      0      0      0       0   
2    2008      3G       0     0   199.0     1      0      0      0       0   
3    2008      3G       0     0   299.0     0      1      0      0       0   
4    2009      3G       1     0   199.0     0      1      0      0       0   
..    ...     ...     ...   ...     ...   ...    ...    ...    ...     ...   
109  2023      15       1     0  1299.0     0      0      0      0       0   
110  2023      15       1     0  1499.0     0      0      0      0       0   
111  2023      15       1     1  1199.0     0      0      0      0       0   
112  2023      15       1     1  1399.0     0      0      0      0       0   
113  2023      15       1     1  1599.0     0      0      0      0       0   

     GB_256  GB_512  GB_1024  
0         0       0        0  
1         0       0        0  
2         0       0        0  
3         0       0        0  
4         0       0        0  
..      ...     ...      ...  
109       0       1        0  
110       0       0        1  
111       1       0        0  
112       0       1        0  
113       0       0        1  

[103 rows x 13 columns]